In [1]:
print('tes')

this is first init for jupyter notebook


# GIT CONFIG


In [2]:
pwd

'/content'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/git-colab-repo/movie-genre-classification

/content/drive/MyDrive/git-colab-repo/movie-genre-classification


In [5]:
!git status

Refresh index: 100% (6/6), done.
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   main.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


pastikan dulu kerja di branch apaa

In [6]:
  !git config --global user.email "asmi.amalia23@gmail.com"
  !git config --global user.name "naamalia23"

In [ ]:
!git add .
!git commit -m 'git config master'
!git push -u origin master